In [1]:
!pip install stop_words pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install --upgrade xlrd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf

from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.metrics import categorical_crossentropy
from keras.callbacks import EarlyStopping

In [4]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
max_words = 300
max_len = 40
num_classes = 1

# Training
epochs = 20
batch_size = 512
print_batch_n = 100

In [6]:
from google.colab import files
uploaded = files.upload()

Saving summer.xls to summer (1).xls


In [7]:
df =  pd.read_excel('summer.xls')
df.head(5)

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


In [8]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

In [9]:
df.drop('Date', axis=1, inplace=True)
df_train, df_test = train_test_split(df, test_size=0.33, random_state=42)

In [10]:
df_train.head()

,Rating,Content
3517,5,Наконец-то исправили эту чушь с неоргинальной ...
12399,5,Удобно в использовании
3833,5,Отлично
13250,5,Класс
17941,5,Удобно


In [11]:
df_test.head()

,Rating,Content
1457,5,Всё отлично!!!
19916,5,Мне нравится. Удобно. Еще ни разу не подвел!
2112,3,Иногда слишком долго приходится ждать пока зап...
15154,5,Отличное приложение. Легко отслеживать свои ра...
18329,5,Все грамотно


In [12]:
df_train['Content'] = df_train['Content'].apply(preprocess_text)
df_test['Content'] = df_test['Content'].apply(preprocess_text)

In [13]:
df_train.head()

,Rating,Content
3517,5,наконецтый исправить чушь снеоргинальный проши...
12399,5,удобно использование
3833,5,отлично
13250,5,класс
17941,5,удобно


In [14]:
df_test.head()

,Rating,Content
1457,5,отлично
19916,5,нравиться удобно разуна подвести
2112,3,приходиться ждать запуститься приложение бввае...
15154,5,отличный приложение легко отслеживать расход о...
18329,5,грамотно


In [15]:
train_corpus = " ".join(df_train["Content"])
train_corpus = train_corpus.lower()

In [16]:
tokens = word_tokenize(train_corpus)
tokens_filtered = [word for word in tokens if word.isalnum()]
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]
tokens_filtered_top[:10]

['приложение',
 'удобно',
 'работать',
 'удобный',
 'отлично',
 'нравиться',
 'хороший',
 'отличный',
 'телефон',
 'супер']

In [17]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}
len(vocabulary)

299

In [18]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [19]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in df_train["Content"]], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in df_test["Content"]], dtype=np.int32)

In [20]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(df_train['Rating']) 
test_enc_labels = le.transform(df_test['Rating'])
le.classes_

array([1, 2, 3, 4, 5])

In [21]:
num_classes = 5
y_train = tf.keras.utils.to_categorical(train_enc_labels, num_classes=num_classes)
y_test = tf.keras.utils.to_categorical(test_enc_labels, num_classes=num_classes)

In [22]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 40, 128)           38400     
                                                                 
 conv1d (Conv1D)             (None, 38, 128)           49280     
                                                                 
 activation (Activation)     (None, 38, 128)           0         
                                                                 
 global_max_pooling1d (Globa  (None, 128)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 10)                1290      
                                                                 
 activation_1 (Activation)   (None, 10)                0         
                                                        

In [23]:
!pip install tensorflow-addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 29.7 MB/s 


In [24]:
import tensorflow_addons as tfa

In [25]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', tfa.metrics.F1Score(num_classes=num_classes, average='macro')])

In [26]:
tensorboard = TensorBoard(log_dir='./logs', write_graph=True, write_images=True)

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard])

Epoch 1/20
25/25 [==============================] - 11s 381ms/step - loss: 1.3383 - accuracy: 0.6745 - f1_score: 0.1953 - val_loss: 1.0421 - val_accuracy: 0.7011 - val_f1_score: 0.1649
Epoch 2/20
25/25 [==============================] - 8s 313ms/step - loss: 0.9406 - accuracy: 0.7054 - f1_score: 0.1755 - val_loss: 0.8819 - val_accuracy: 0.7213 - val_f1_score: 0.2398
Epoch 3/20
25/25 [==============================] - 7s 287ms/step - loss: 0.7834 - accuracy: 0.7536 - f1_score: 0.3056 - val_loss: 0.7773 - val_accuracy: 0.7430 - val_f1_score: 0.3052
Epoch 4/20
25/25 [==============================] - 7s 283ms/step - loss: 0.6969 - accuracy: 0.7706 - f1_score: 0.3300 - val_loss: 0.7241 - val_accuracy: 0.7408 - val_f1_score: 0.3125
Epoch 5/20
25/25 [==============================] - 4s 158ms/step - loss: 0.6563 - accuracy: 0.7759 - f1_score: 0.3572 - val_loss: 0.7051 - val_accuracy: 0.7523 - val_f1_score: 0.3365
Epoch 6/20
25/25 [==============================] - 4s 156ms/step - loss: 0.634

Предобученный слой

In [27]:
!wget http://vectors.nlpl.eu/repository/20/220.zip

--2022-11-02 20:00:38--  http://vectors.nlpl.eu/repository/20/220.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.181
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 638171816 (609M) [application/zip]
Saving to: ‘220.zip’

220.zip             100%[===================>] 608.61M  26.9MB/s    in 23s     

2022-11-02 20:01:02 (26.1 MB/s) - ‘220.zip’ saved [638171816/638171816]



In [28]:
!unzip 220.zip

Archive:  220.zip
  inflating: meta.json               
  inflating: model.bin               
  inflating: model.txt               
  inflating: README                  


In [29]:
!pip install -U gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.1 MB 67.0 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [30]:
import gensim

In [31]:
word_model = gensim.models.KeyedVectors.load_word2vec_format('model.bin', binary=True)

In [32]:
len(word_model)

249333

In [33]:
word_model_matrix = [word_model[i][:128] for i in range(300)]

In [34]:
initializer = tf.keras.initializers.Constant(word_model_matrix)

model_w = Sequential()
model_w.add(Embedding(input_dim=max_words, output_dim=128, embeddings_initializer =initializer, input_length=max_len))
model_w.add(Conv1D(128, 3))
model_w.add(Activation("relu"))
model_w.add(GlobalMaxPool1D())
model_w.add(Dense(10))
model_w.add(Activation("relu"))
model_w.add(Dense(num_classes))
model_w.add(Activation('softmax'))
model_w.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 40, 128)           38400     
                                                                 
 conv1d_1 (Conv1D)           (None, 38, 128)           49280     
                                                                 
 activation_3 (Activation)   (None, 38, 128)           0         
                                                                 
 global_max_pooling1d_1 (Glo  (None, 128)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_2 (Dense)             (None, 10)                1290      
                                                                 
 activation_4 (Activation)   (None, 10)                0         
                                                      

In [35]:
model_w.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', tfa.metrics.F1Score(num_classes=num_classes, average='macro')])

In [36]:
tensorboard = TensorBoard(log_dir='./logs', write_graph=True, write_images=True)

history_w = model_w.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard])

Epoch 1/20
25/25 [==============================] - 8s 273ms/step - loss: 1.3159 - accuracy: 0.6261 - f1_score: 0.2041 - val_loss: 0.9494 - val_accuracy: 0.6895 - val_f1_score: 0.2038
Epoch 2/20
25/25 [==============================] - 10s 389ms/step - loss: 0.9138 - accuracy: 0.6916 - f1_score: 0.2420 - val_loss: 0.8978 - val_accuracy: 0.6895 - val_f1_score: 0.2217
Epoch 3/20
25/25 [==============================] - 8s 301ms/step - loss: 0.8587 - accuracy: 0.7001 - f1_score: 0.2368 - val_loss: 0.8799 - val_accuracy: 0.7032 - val_f1_score: 0.2311
Epoch 4/20
25/25 [==============================] - 8s 308ms/step - loss: 0.8145 - accuracy: 0.7063 - f1_score: 0.2407 - val_loss: 0.8392 - val_accuracy: 0.7004 - val_f1_score: 0.2293
Epoch 5/20
25/25 [==============================] - 5s 195ms/step - loss: 0.7789 - accuracy: 0.7127 - f1_score: 0.2526 - val_loss: 0.8255 - val_accuracy: 0.6996 - val_f1_score: 0.2334
Epoch 6/20
25/25 [==============================] - 4s 156ms/step - loss: 0.759

In [37]:
score = model_w.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)

14/14 [==============================] - 1s 44ms/step - loss: 0.7565 - accuracy: 0.7454 - f1_score: 0.3221


Модели показали примерно одинаковый результат